In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer, KNNImputer

In [2]:
train = pd.read_csv('train.csv')
train.head()

,id,0,1,2,3,4,5,6,7,8,...,81,82,83,84,85,86,87,88,89,label
0,0,-166.824782,17.033603,113.694779,NaN,-182.215604,-2.521714,-199.901124,NaN,-45.708388,...,-332.452466,85.085788,-13.024603,-9.562681,83.766392,85.735232,56.783683,-38.675632,150.462556,0
1,1,365.475478,-26.553938,236.325629,-52.226619,-178.108750,9.660407,208.059278,-72.996101,-204.824642,...,-177.413509,86.435633,219.232681,210.197297,310.312671,57.361344,-154.756712,-32.910450,-174.949022,1
2,2,256.087325,142.746874,225.408770,-137.328993,-116.358770,103.970800,-188.420302,-32.550649,NaN,...,-577.352793,226.505005,-150.845434,-59.454808,59.194336,95.909930,-45.067073,15.599768,-58.092245,9
3,3,256.717078,130.483910,310.188561,45.661678,-218.639113,-236.996842,-199.403190,-253.164302,-63.168319,...,227.910817,-70.595150,459.307503,49.871418,-249.139920,88.601608,23.564391,-32.892407,-320.543956,3
4,4,-241.955610,-83.173692,301.710460,-91.500567,NaN,121.229830,-258.974149,-322.076770,-49.659415,...,328.431952,92.227399,467.847176,-40.249718,57.830269,-146.688066,220.323157,34.831294,24.678597,6


In [3]:
test = pd.read_csv('test.csv')
test.head()

,id,0,1,2,3,4,5,6,7,8,...,80,81,82,83,84,85,86,87,88,89
0,0,NaN,NaN,78.105602,NaN,27.263848,NaN,86.017175,50.424767,-61.915767,...,-74.031231,479.785984,54.962311,-343.979735,-93.251990,33.010356,-54.498837,-39.568282,-111.535203,-8.233110
1,1,30.000909,NaN,691.954480,-46.286694,NaN,NaN,-23.244682,-291.611654,-55.534847,...,655.078687,441.989100,-138.524768,-124.655609,61.867380,-90.086187,-18.298289,-169.294359,-196.780912,-7.127010
2,2,87.057436,106.860293,NaN,46.245733,-204.271635,-236.349492,-132.876512,32.830881,190.534127,...,180.217779,31.193760,134.389222,-208.634289,136.196460,49.124821,-41.453356,168.939329,-70.029444,-27.508850
3,3,133.683072,140.666470,NaN,-43.268632,NaN,93.804596,4.737805,621.660742,215.324634,...,273.187794,310.190438,-61.420497,156.520005,170.077972,81.772007,99.166807,266.081272,-185.359210,-66.228673
4,4,52.002690,24.682843,250.992064,76.731366,-1.518029,-8.136746,NaN,135.728587,48.296441,...,459.397881,-186.687745,44.344978,-48.723744,52.798684,49.864260,-94.273259,167.404452,-48.654228,-25.918759


In [4]:
test.shape, train.shape

((10000, 91), (50000, 92))

In [5]:
y_pd = train['label']
X_pd = train.drop(columns=['label', 'id'])

X_clear = X_pd.dropna()
X_clear.shape, X_pd.shape

((1779, 90), (50000, 90))

In [6]:
X = np.array(X_pd)
y = np.array(y_pd)
X.shape, y.shape

((50000, 90), (50000,))

In [7]:
X_for_test_pd = test.drop(columns=['id'])
X_for_test = np.array(X_for_test_pd)
X_for_test.shape

(10000, 90)

Попробуем kNN с заполнением неизвестных значений по 'mean' или 'most_frequent'

In [8]:
for strat in ['mean', 'most_frequent']:
    for k in [1, 2]:
        for neigh in [1,2,3]:
            for weight in ['uniform', 'distance']:
                my_kf = KFold(n_splits = 3)
                my_imp = SimpleImputer(strategy = strat, missing_values=np.nan)
                X_fixed = my_imp.fit_transform(X)
                for train_index, test_index in my_kf.split(X_fixed):
                    X_train, X_test = X_fixed[train_index], X_fixed[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=neigh, p = k, weights=weight)
                    knnClassifier.fit(X_train, y_train)
                    tmp_res = knnClassifier.predict(X_test)
                    accur = (tmp_res - y_test != 0).sum()/len(y_test)
                    print('{0} with {1} {2} {3} {4}'.format(accur, strat, k, neigh, weight))
                    


0.6141477170456591 with mean 1 1 uniform
0.32741345173096537 with mean 1 1 uniform
0.33055322212888516 with mean 1 1 uniform
0.6141477170456591 with mean 1 1 distance
0.32741345173096537 with mean 1 1 distance
0.33055322212888516 with mean 1 1 distance
0.618407631847363 with mean 1 2 uniform
0.18173636527269454 with mean 1 2 uniform
0.1813272530901236 with mean 1 2 uniform
0.6141477170456591 with mean 1 2 distance
0.32741345173096537 with mean 1 2 distance
0.33055322212888516 with mean 1 2 distance
0.6139077218455631 with mean 1 3 uniform
0.25499490010199793 with mean 1 3 uniform
0.25465018600744027 with mean 1 3 uniform
0.6128277434451311 with mean 1 3 distance
0.25445491090178196 with mean 1 3 distance
0.2518900756030241 with mean 1 3 distance
0.6148077038459231 with mean 2 1 uniform
0.33095338093238136 with mean 2 1 uniform
0.33049321972878914 with mean 2 1 uniform
0.6148077038459231 with mean 2 1 distance
0.33095338093238136 with mean 2 1 distance
0.33049321972878914 with mean 2 1 

Попробуем теперь использовать most_frequent с большим количеством соседей и с метрикой p = 2 (наилушие показатели)

In [9]:
k = 2
strat = 'most_frequent'
my_imp = SimpleImputer(strategy = strat, missing_values=np.nan)
X_fixed = my_imp.fit_transform(X)

for neigh in range(5, 15):
    for weight in ['uniform', 'distance']:
        my_kf = KFold(n_splits = 3)
        for train_index, test_index in my_kf.split(X_fixed):
            X_train, X_test = X_fixed[train_index], X_fixed[test_index]
            y_train, y_test = y[train_index], y[test_index]
            knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=neigh, p = k, weights=weight)
            knnClassifier.fit(X_train, y_train)
            tmp_res = knnClassifier.predict(X_test)
            accur = (tmp_res - y_test != 0).sum()/len(y_test)
            print('{0} with {1} {2} {3} {4}'.format(accur, strat, k, neigh, weight))

0.631427371452571 with most_frequent 2 5 uniform
0.24623507529849403 with most_frequent 2 5 uniform
0.24282971318852753 with most_frequent 2 5 uniform
0.629867402651947 with most_frequent 2 5 distance
0.254034919301614 with most_frequent 2 5 distance
0.24780991239649586 with most_frequent 2 5 distance
0.6313673726525469 with most_frequent 2 6 uniform
0.19379612407751845 with most_frequent 2 6 uniform
0.18912756510260412 with most_frequent 2 6 uniform
0.629267414651707 with most_frequent 2 6 distance
0.24371512569748605 with most_frequent 2 6 distance
0.23976959078363136 with most_frequent 2 6 distance
0.630047399052019 with most_frequent 2 7 uniform
0.21035579288414233 with most_frequent 2 7 uniform
0.2093483739349574 with most_frequent 2 7 uniform
0.628967420651587 with most_frequent 2 7 distance
0.21827563448731024 with most_frequent 2 7 distance
0.2154686187447498 with most_frequent 2 7 distance
0.630047399052019 with most_frequent 2 8 uniform
0.17705645887082258 with most_frequent 

Заменим most_frequent на подбор пропущенных значений по kNN

In [38]:
k = 2
strat = 'kNN'

for neigh in range(2, 20, 2):
    for weight in ['uniform', 'distance']:
        knn_imp = KNNImputer(n_neighbors=neigh, weights=weight)
        X_fixed = knn_imp.fit_transform(X)
        my_kf = KFold(n_splits = 5)
        for train_index, test_index in my_kf.split(X_fixed):
            X_train, X_test = X_fixed[train_index], X_fixed[test_index]
            y_train, y_test = y[train_index], y[test_index]
            knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=neigh, p = k, weights=weight)
            knnClassifier.fit(X_train, y_train)
            tmp_res = knnClassifier.predict(X_test)
            accur = (tmp_res - y_test != 0).sum()/len(y_test)
            print('{0} with {1} {2} {3} {4}'.format(accur, strat, k, neigh, weight))

0.9955 with kNN 2 2 uniform
0.2142 with kNN 2 2 uniform
0.2085 with kNN 2 2 uniform
0.2147 with kNN 2 2 uniform
0.211 with kNN 2 2 uniform
0.9918 with kNN 2 2 distance
0.3263 with kNN 2 2 distance
0.3304 with kNN 2 2 distance
0.3257 with kNN 2 2 distance
0.3291 with kNN 2 2 distance
0.995 with kNN 2 4 uniform
0.1749 with kNN 2 4 uniform
0.1744 with kNN 2 4 uniform
0.1756 with kNN 2 4 uniform
0.172 with kNN 2 4 uniform
0.9936 with kNN 2 4 distance
0.2329 with kNN 2 4 distance
0.2378 with kNN 2 4 distance
0.234 with kNN 2 4 distance
0.2297 with kNN 2 4 distance
0.9938 with kNN 2 6 uniform
0.1522 with kNN 2 6 uniform
0.1475 with kNN 2 6 uniform
0.1503 with kNN 2 6 uniform
0.1454 with kNN 2 6 uniform
0.9926 with kNN 2 6 distance
0.1906 with kNN 2 6 distance
0.1849 with kNN 2 6 distance
0.1848 with kNN 2 6 distance
0.1818 with kNN 2 6 distance
0.9937 with kNN 2 8 uniform
0.1358 with kNN 2 8 uniform
0.1315 with kNN 2 8 uniform
0.1392 with kNN 2 8 uniform
0.131 with kNN 2 8 uniform
0.9929 wit

Попроубем использовать заполнение пропущенных значений по kNN, а предсказание -- по наивному Байесу с гауссовским распределением.

Возьмем наивный Байес и заполнение по kNN distance 6.
Данные, которые по которым нам нужно предсказать класс, тоже дополним по kNN 6 dist.

In [25]:
knn_imp = KNNImputer(n_neighbors=neigh, weights=weight)
X_fixed = knn_imp.fit_transform(X)
X_for_test_fixed = knn_imp.fit_transform(X_for_test)

my_clf = GaussianNB()
my_clf.fit(X_fixed, y)

my_labels = my_clf.predict(X_for_test_fixed)
my_labels

array([1, 8, 9, ..., 5, 3, 3], dtype=int64)

In [29]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels})
result.head(), result.shape

(   id  label
 0   0      1
 1   1      8
 2   2      9
 3   3      9
 4   4      6,
 (10000, 2))

In [31]:
result.to_csv('DoynichenkoMaxim.csv', index=False)

Возьмем kNN 6 dist

In [32]:
knn_imp = KNNImputer(n_neighbors=neigh, weights=weight)
X_fixed = knn_imp.fit_transform(X)
X_for_test_fixed = knn_imp.fit_transform(X_for_test)


knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=6, p = 2, weights='distance')
knnClassifier.fit(X_fixed, y)
my_labels = knnClassifier.predict(X_for_test_fixed)
my_labels

array([1, 8, 9, ..., 5, 3, 3], dtype=int64)

In [33]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels})
result.to_csv('DoynichenkoMaxim.csv', index=False)

Еще стоит попробовать kNN 4 dist

In [36]:
knn_imp = KNNImputer(n_neighbors=4, weights='distance')
X_fixed = my_imp.fit_transform(X)
X_for_test_fixed = my_imp.fit_transform(X_for_test)


knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=4, p = 2, weights='distance')
knnClassifier.fit(X_fixed, y)
my_labels = knnClassifier.predict(X_for_test_fixed)
my_labels

array([1, 8, 9, ..., 5, 3, 3], dtype=int64)

In [37]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels})
result.to_csv('DoynichenkoMaxim.csv', index=False)

Попробуем SVM

Заготовим сразу наборы с заполнением пропусков по kNN, так как их вычисление занимает очень много времени

In [12]:
knn_imp = KNNImputer(n_neighbors=4, weights='distance')
X_train_kNN4_fixed = knn_imp.fit_transform(X)
print('Done for kNN 4')

knn_imp = KNNImputer(n_neighbors=6, weights='distance')
X_train_kNN6_fixed = knn_imp.fit_transform(X)
print('Done for kNN 6')

knn_imp = KNNImputer(n_neighbors=8, weights='distance')
X_train_kNN8_fixed = knn_imp.fit_transform(X)
print('Done for kNN 8')


Done for kNN 4
Done for kNN 6
Done for kNN 8


In [14]:
from sklearn.naive_bayes import GaussianNB

k = 2
strat = 'kNN & Bayes'

for data_imp in [X_train_kNN4_fixed, X_train_kNN6_fixed, X_train_kNN8_fixed]:
    print('=======')
    my_kf = KFold(n_splits = 5)
    for train_index, test_index in my_kf.split(data_imp):
        X_train, X_test = data_imp[train_index], data_imp[test_index]
        y_train, y_test = y[train_index], y[test_index]

        my_clf = GaussianNB()
        my_clf.fit(X_train, y_train)

        tmp_res = my_clf.predict(X_test)

        accur = (tmp_res - y_test != 0).sum()/len(y_test)
        print('{0}'.format(accur))


0.9896
0.1606
0.1472
0.1591
0.1578
0.9894
0.1613
0.1484
0.16
0.1574
0.9898
0.1616
0.1481
0.1604
0.1572


In [31]:
from sklearn.svm import SVC

strat = 'kNN & SVM'
weight = 'distance'

for data_imp in [X_train_kNN4_fixed, X_train_kNN6_fixed, X_train_kNN8_fixed]:
    print('=======')

    my_kf = KFold(n_splits = 5)
    for train_index, test_index in my_kf.split(data_imp):
        X_train, X_test = data_imp[train_index], data_imp[test_index]
        y_train, y_test = y[train_index], y[test_index]

        my_clf = SVC()
        my_clf.fit(X_train, y_train)

        tmp_res = my_clf.predict(X_test)

        accur = (tmp_res - y_test != 0).sum()/len(y_test)
        print('{0} '.format(accur))

0.9972 
0.0311 
0.0294 
0.0307 
0.0245 
0.9973 
0.0318 
0.0291 
0.0318 
0.0244 
0.9972 
0.0322 
0.0299 
0.0313 
0.0248 


Попробуем kNN и в заполнении пропущенных значений, и в определении класса

In [16]:
k = 2
strat = 'kNN'
weight = 'distance'

for neigh in range(4, 14, 2):
    for data_imp in [X_train_kNN4_fixed, X_train_kNN6_fixed, X_train_kNN8_fixed]:
        my_kf = KFold(n_splits = 3)
        for train_index, test_index in my_kf.split(data_imp):
            X_train, X_test = data_imp[train_index], data_imp[test_index]
            y_train, y_test = y[train_index], y[test_index]
            knnClassifier = sk.neighbors.KNeighborsClassifier(n_neighbors=neigh, p = k, weights=weight)
            knnClassifier.fit(X_train, y_train)
            tmp_res = knnClassifier.predict(X_test)
            accur = (tmp_res - y_test != 0).sum()/len(y_test)
            print('{0} with {1} {2} {3} {4}'.format(accur, strat, k, neigh, weight))

0.6116877662446751 with kNN 2 4 distance
0.24731505369892604 with kNN 2 4 distance
0.2485299411976479 with kNN 2 4 distance
0.6116277674446511 with kNN 2 4 distance
0.24605507889842204 with kNN 2 4 distance
0.24768990759630385 with kNN 2 4 distance
0.6113877722445551 with kNN 2 4 distance
0.24563508729825403 with kNN 2 4 distance
0.2487699507980319 with kNN 2 4 distance
0.612227755444891 with kNN 2 6 distance
0.20141597168056638 with kNN 2 6 distance
0.19890795631825273 with kNN 2 6 distance
0.6124677506449872 with kNN 2 6 distance
0.2011159776804464 with kNN 2 6 distance
0.19878795151806072 with kNN 2 6 distance
0.6121677566448671 with kNN 2 6 distance
0.1990760184796304 with kNN 2 6 distance
0.19776791071642866 with kNN 2 6 distance
0.6125277494450111 with kNN 2 8 distance
0.16643667126657466 with kNN 2 8 distance
0.1643465738629545 with kNN 2 8 distance
0.6125877482450351 with kNN 2 8 distance
0.16511669766604667 with kNN 2 8 distance
0.16392655706228249 with kNN 2 8 distance
0.6122

In [ ]:
X_train_kNN4_fixed.to_csv('train_x_4.csv', index = False)
X_train_kNN6_fixed.to_csv('train_x_6.csv', index = False)
X_train_kNN8_fixed.to_csv('train_x_8.csv', index = False)

knn_imp = KNNImputer(n_neighbors=4, weights='distance')
X_test_kNN4_fixed = knn_imp.fit_transform(X_for_test)
print('Done for kNN 4 [test]')

knn_imp = KNNImputer(n_neighbors=6, weights='distance')
X_test_kNN6_fixed = knn_imp.fit_transform(X_for_test)
print('Done for kNN 6 [test]')

knn_imp = KNNImputer(n_neighbors=8, weights='distance')
X_test_kNN8_fixed = knn_imp.fit_transform(X_for_test)
print('Done for kNN 8 [test]')

X_test_kNN4_fixed.to_csv('test_x_4.csv', index = False)
X_test_kNN6_fixed.to_csv('test_x_6.csv', index = False)
X_test_kNN8_fixed.to_csv('test_x_8.csv', index = False)

SVM + kNN обработка. Судя по тестам, лучший вариант -- 4 соседа

In [18]:
knn_imp = KNNImputer(n_neighbors=4, weights='distance')
X_test_kNN4_fixed = knn_imp.fit_transform(X_for_test)

print('test_transformed')



test_transformed


my_clf = SVC()
my_clf.fit(X_train_kNN4_fixed, y)

my_labels = my_clf.predict(X_test_kNN4_fixed)
my_labels

In [22]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels})
result.to_csv('DoynichenkoMaxim.csv', index=False)

Попробуем SVM c заполнением по 'mean'

In [26]:
from sklearn.svm import SVC

my_imp = SimpleImputer(strategy='mean')
X_mean_fixed_train = my_imp.fit_transform(X)
X_mean_fixed_test = my_imp.fit_transform(X_for_test)

my_clf = SVC()
my_clf.fit(X_mean_fixed_train, y)
my_labels = my_clf.predict(X_mean_fixed_test)
my_labels


array([1, 8, 9, ..., 5, 3, 3], dtype=int64)

In [27]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels})
result.to_csv('DoynichenkoMaxim.csv', index=False)

In [28]:
from sklearn.svm import SVC


my_clf = SVC()
my_clf.fit(X_train_kNN4_fixed, y)
my_labels2 = my_clf.predict(X_test_kNN4_fixed)
my_labels2



array([1, 8, 9, ..., 5, 3, 3], dtype=int64)

In [29]:
(my_labels - my_labels2 != 0).sum()/len(my_labels)

0.0104

In [30]:
result = pd.DataFrame({'id': test['id'], 'label': my_labels2})
result.to_csv('DoynichenkoMaxim.csv', index=False)

In [32]:
from sklearn.svm import SVC

weight = 'distance'

for strat in ['mean', 'most_frequent']:
    print('=======')
    my_imp = SimpleImputer(strategy=strat)
    X_fixed = my_imp.fit_transform(X)
    my_kf = KFold(n_splits = 5)
    for train_index, test_index in my_kf.split(X_fixed):
        X_train, X_test = X_fixed[train_index], X_fixed[test_index]
        y_train, y_test = y[train_index], y[test_index]

        my_clf = SVC()
        my_clf.fit(X_train, y_train)

        tmp_res = my_clf.predict(X_test)

        accur = (tmp_res - y_test != 0).sum()/len(y_test)
        print('{0} '.format(accur))

0.9975 
0.0344 
0.0323 
0.0339 
0.0269 
0.9964 
0.0529 
0.0477 
0.0496 
0.0426 


В итоге лучшие показатели в Public-тесте были достигнуты на SVC с заполнением пустых значений с помощью SimpleImputer со стратегией 'mean'. Скорее всего при использовании SVC и заполнения с помощью kNN с 4 соседями и весами 'distance' точность на Public-тесте была бы выше, но него его отправку уже не хватило сабмита (лимит 5 сабмитов в сутки).